<a href="https://colab.research.google.com/github/gabriel1628/Meeting-Scheduling-Assistant/blob/main/chat_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets -q
# !pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import pipeline
from datasets import load_from_disk
import os
import torch
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(userdata.get('HF_TOKEN'))

In [ ]:
ds = load_from_disk("/content/drive/MyDrive/Colab Notebooks/AI Meeting Assistant/data")
ds

Dataset({
    features: ['text', 'label'],
    num_rows: 264
})

In [ ]:
ds[0]

{'text': 'Arrange a meeting with Florian this week or next week in the afternoon',
 'label': 0}

In [ ]:
# Define the cache directory in your Google Drive
cache_dir = '/content/drive/My Drive/huggingface_models'

# Create the directory if it doesn't exist
os.makedirs(cache_dir, exist_ok=True)

# Set the cache directory for the transformers library
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME'] = cache_dir
os.environ['HF_HUB_CACHE'] = cache_dir

In [ ]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
!ls /root/.cache/huggingface/hub/ -a

.	models--Helsinki-NLP--opus-mt-en-fr	   models--meta-llama--Llama-3.2-3B-Instruct
..	models--Helsinki-NLP--opus-mt-fr-en	   models--openai-community--gpt2
.locks	models--meta-llama--Llama-3.2-1B-Instruct  version.txt


In [ ]:
now = datetime.now()
meeting_request = "Arrange a meeting with Florian this week or next week in the afternoon"
prompt = f"""
Here is a meeting request : {meeting_request}
Knowing today is {now.strftime("%A %d of %B %Y")}, suggest the earliest 3 non-overlapping time slots that satisfies
the constraints of the request. Answer only with a numbered list of the time slots in ISO format.
"""
chat = [
    {"role": "system", "content": "You are a meeting assistant who schedules meetings based on the user's request. Meetings must be scheduled on a week day and between 8 am and 5 pm."},
    {"role": "user", "content": prompt}
]

In [ ]:
%%time
response = pipe(chat, max_new_tokens=75)
print(response[0]['generated_text'][-1]['content'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Here are the suggested time slots:

1. 2024-12-16T14:00:00
2. 2024-12-17T14:00:00
3. 2024-12-18T14:00:00
CPU times: user 2.53 s, sys: 118 ms, total: 2.65 s
Wall time: 2.7 s


In [ ]:
chat = response[0]['generated_text']
now = datetime.now()
prompt = "Suggest 3 other time slots that satisfies the constraints of the request"
chat.append(
    {"role": "user", "content": prompt}
)
response = pipe(chat, max_new_tokens=75)
print(response[0]['generated_text'][-1]['content'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Here are the additional suggested time slots:

1. 2024-12-17T16:00:00
2. 2024-12-18T16:00:00
3. 2024-12-19T16:00:00


In [ ]:
slot_1 = response[0]["generated_text"][-1]["content"].split("1. ")[1].split("\n")[0]
slot_2 = response[0]["generated_text"][-1]["content"].split("2. ")[1].split("\n")[0]
slot_3 = response[0]["generated_text"][-1]["content"].split("3. ")[1].split("\n")[0]

In [ ]:
print(slot_1)
print(slot_2)
print(slot_3)

2024-12-17T16:00:00
2024-12-18T16:00:00
2024-12-19T16:00:00


In [ ]:
datetime.fromisoformat(slot_1)

datetime.datetime(2024, 12, 13, 13, 0)

In [ ]:
chat = response[0]['generated_text']
now = datetime.now()
prompt = "Suggest 3 other time slots with different times each"
chat.append(
    {"role": "user", "content": prompt}
)
response = pipe(chat, max_new_tokens=75)
print(response[0]['generated_text'][-1]['content'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Here are the additional suggested time slots:

1. 2024-12-16T17:00:00
2. 2024-12-18T14:30:00
3. 2024-12-17T13:30:00
